




















































































































































































## Deep Learning Project

## Group Partners

### Muhammad Salman Iqbal 21i-2302 MS(DS)

### Muhammad Usman Khan 21i-2163 MS(AI)

### Ansa Liaqat 21i-2195 MS(AI)

In [11]:
# Write own Custom dataset Class using pytorch Dataset class and torchvision which takes the root folder as input and return triplet of images
# Dataset class is used to load the data and return the triplet of image

# import the necessary packages
from torch.utils.data import Dataset
import os
import numpy as np
from skimage import io, transform
import torch
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader 
import math
from torch.nn.modules.distance import PairwiseDistance


In [12]:
# Check GPU availability
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


cuda:0


In [46]:
#from google.colab import drive
#drive.mount('/content/drive')

In [14]:
#cp "/content/drive/MyDrive/lfw.zip" .

In [15]:
#!unzip "lfw.zip"

In [16]:
# define the custom dataset class
class TripletDataset(Dataset):
    def __init__(self, dataset_path, transform=None ,triplets=None, triplet = None):
        # initialize the list of image paths and labels
        self.image_paths = self.__image_paths__(dataset_path)
        self.labels = self.__getLabels__(dataset_path)
        self.transform = transform
        if(triplets == None or len(triplets) == 0 or len(triplet)==0):
            self.triplets = self.generate_triplets(dataset_path)
        else:
            
            self.triplets = self.getList(triplets,triplet)
        

    def getList(self ,triplets, triplet):
        tri=[]
        print(len(triplet))
        print(len(triplets))
        for i in range(len(triplet)):
            tri.append(triplets[triplet[i]])

        return tri

    def __getTriplets__(self):
        return self.triplets

    def __image_paths__(self, dataset_path):
        image_paths = []
        for l in os.listdir(dataset_path):
            path = os.path.sep.join([dataset_path, l])
            image_paths.extend(list(map(lambda x: os.path.sep.join([path, x]), os.listdir(path))))
        return image_paths   

    def __getLabels__(self ,dataset_path):
            labels =[]
            for l in os.listdir(dataset_path):
                path = os.path.sep.join([dataset_path, l])
                for i in range(0 ,len(os.listdir(path))):
                    labels.append(l)
            return labels    

    def __getitem__(self, idx):
        # load the image from disk
        anchor_image, pos_image, neg_image ,pos_class, pos_class, neg_class= self.triplets[idx]
       
        anchor_image = io.imread(anchor_image)
        pos_image = io.imread(pos_image)
        neg_image = io.imread(neg_image)
        pos_class = self.labels.index(pos_class)
        neg_class = self.labels.index(neg_class)
        pos_class = torch.from_numpy(np.array([pos_class]).astype('long'))
        neg_class = torch.from_numpy(np.array([neg_class]).astype('long'))

        # apply the transformations
        if self.transform:
            anchor_image = self.transform(anchor_image)
            pos_image = self.transform(pos_image)
            neg_image = self.transform(neg_image)
        
            

        # return a tuple of the images and the labels
        return idx, anchor_image, pos_image, neg_image, pos_class, neg_class

    def __len__(self):
        return len(self.triplets)

    def generate_triplets(self , datset_path):
        '''
              - randomly choose anchor, positive and negative images for triplet loss
              - anchor and positive images in pos_class
              - negative image in neg_class
              - at least, two images needed for anchor and positive images in pos_class
              - negative image should have different class as anchor and positive images by definition
            '''
        # Get Label for each image indivisually
        triplets=[]
        lab = list(set(self.labels))
        for i in range(0,6600):
            pos_class = np.random.choice(lab)
            neg_class = np.random.choice(lab)
            path = os.path.sep.join([datset_path , pos_class])
            negpath = os.path.sep.join([datset_path , neg_class])
            anchor_image , neg_image , pos_image='' ,'' ,''
            while len(os.listdir(path)) < 2:
                pos_class = lab[np.random.randint(0, len(lab))]
                path = os.path.sep.join([datset_path , pos_class])
            while len(os.listdir(negpath)) < 1:
                neg_class = lab[np.random.randint(0, len(lab))]
                while neg_class == pos_class:
                    neg_class = np.random.randint(0, len(lab))
                negpath = os.path.sep.join([datset_path , neg_class])
            
           

            # Get random image from pos_class
            pos_image =(np.random.choice(os.listdir(path)))
            pos_image = os.path.sep.join([datset_path ,pos_class , pos_image])
            anchor_image = np.random.choice(os.listdir(path))
            anchor_image = os.path.sep.join([datset_path ,pos_class, anchor_image])
            # Get random image from neg_class
            neg_image = (np.random.choice(os.listdir(negpath)))
            neg_image = os.path.sep.join([datset_path,neg_class , neg_image])
            triplets.append((anchor_image, pos_image, neg_image, pos_class, pos_class, neg_class))

            
        
        print(len(triplets))
        return triplets
                


# define the transform(s) to apply to the data
data_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(256),
    transforms.CenterCrop(224),
   
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])







In [17]:

triplets=[]

In [18]:
# saimes network 
# import libraries for nn
import torch.nn as nn
import torch.nn.functional as F

# 


class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        self.cnn1 = nn.Sequential(
            nn.ReflectionPad2d(1),
            nn.Conv2d(3, 4, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(4),
            nn.Dropout2d(p=0.2),
            nn.Conv2d(4, 8, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(8),
            nn.Dropout2d(p=0.2),
            nn.MaxPool2d(2),
        )
        # Convert image to vector of size 128 must be fully connected layer
        self.fc1 = nn.Sequential(
            nn.Linear(98568,1000),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(1000),
            nn.Dropout(p=0.5),
            nn.Linear(1000,128),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(128),
            nn.Dropout(p=0.5),
        )


    def forward_once(self, x):
        output = self.cnn1(x)
        output = output.view(output.size()[0], -1)
        output = self.fc1(output)
        return output

    def forward(self, input1, input2 , input3):
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)
        output3 = self.forward_once(input3)
        return output1, output2 , output3

In [19]:
# Loss Function
import torch



class TripletLoss(torch.nn.Module):

    def __init__(self, margin):
        super(TripletLoss, self).__init__()
        self.margin = margin
        self.pdist = PairwiseDistance(2)

    def forward(self,idx, anchor, positive, negative):
        pos_dist = self.pdist.forward(anchor, positive)
        neg_dist = self.pdist.forward(anchor, negative)

        hinge_dist = torch.clamp(self.margin + pos_dist - neg_dist, min=0.0)
        for i in range(len(hinge_dist)):
            if hinge_dist[i] == 0:
                continue
            else:

                triplet.append(idx[i].item())
                
        print(hinge_dist)
        loss = torch.mean(hinge_dist)
        return loss

In [20]:
# import Variable
from torch.autograd import Variable
# initialize the siamese network
model = SiameseNetwork()
# to device
model = model.to(device)

# initialize the criterion and optimizer
# criterion = nn.TripletMarginLoss(margin=2.0)
criterion = TripletLoss(2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001 )




In [21]:
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: 
            continue
        param = parameter.numel()
        table.add_row([name, param])
        total_params+=param
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

count_parameters(model)

+---------------+------------+
|    Modules    | Parameters |
+---------------+------------+
| cnn1.1.weight |    108     |
|  cnn1.1.bias  |     4      |
| cnn1.3.weight |     4      |
|  cnn1.3.bias  |     4      |
| cnn1.5.weight |    288     |
|  cnn1.5.bias  |     8      |
| cnn1.7.weight |     8      |
|  cnn1.7.bias  |     8      |
|  fc1.0.weight |  98568000  |
|   fc1.0.bias  |    1000    |
|  fc1.2.weight |    1000    |
|   fc1.2.bias  |    1000    |
|  fc1.4.weight |   128000   |
|   fc1.4.bias  |    128     |
|  fc1.6.weight |    128     |
|   fc1.6.bias  |    128     |
+---------------+------------+
Total Trainable Params: 98699816


98699816

In [22]:
dataset = TripletDataset(dataset_path="lfw", transform=data_transforms )

6600


In [23]:

n_epochs = 5
triplet=[]
from torch.nn.modules.distance import PairwiseDistance
model=model.to(device)
i=0
for epoch in range(0,n_epochs):
    
    dataset = TripletDataset(dataset_path="lfw", transform=data_transforms ,triplets=dataset.__getTriplets__(), triplet=triplet)
    data = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=0 , drop_last=True )
    triplet=[]
    dataIter = iter(data)
    for idx ,anchor_image, pos_image, neg_image, pos_class, neg_class in dataIter:
        anchor_image, pos_image, neg_image = Variable(anchor_image), Variable(pos_image), Variable(neg_image)
        pos_class, neg_class = Variable(pos_class), Variable(neg_class)
        anchor_image, pos_image, neg_image = anchor_image.to(device), pos_image.to(device), neg_image.to(device)
        optimizer.zero_grad()
        anchor_image_out, pos_output2 , neg_output3 = model(anchor_image, pos_image , neg_image)
        print(idx)
        loss = criterion(idx,anchor_image_out, pos_output2, neg_output3)

        loss.backward()
        optimizer.step()
        i=i+1
        if i%100==0:

            print(f"Epoch: {epoch}/{n_epochs}")
            print(f"Loss: {loss}")
           
        

6600
tensor([6416, 4142,   55, 6144])
tensor([4.2736, 3.0544, 2.9488, 2.9489], device='cuda:0',
       grad_fn=<ClampBackward1>)
tensor([5345,  542, 1264, 3394])
tensor([1.1086, 0.0000, 0.0000, 4.6612], device='cuda:0',
       grad_fn=<ClampBackward1>)
tensor([  68, 4945, 4507, 5538])
tensor([1.4257, 0.0000, 0.2776, 0.0000], device='cuda:0',
       grad_fn=<ClampBackward1>)
tensor([6479, 6542, 6234, 2399])
tensor([0.0000, 2.3941, 5.2701, 5.6114], device='cuda:0',
       grad_fn=<ClampBackward1>)
tensor([5879, 5361, 5243, 4798])
tensor([1.4110, 3.5254, 0.0059, 0.7710], device='cuda:0',
       grad_fn=<ClampBackward1>)
tensor([2828, 3170, 3785, 3670])
tensor([0.0000, 0.0000, 3.0924, 2.4237], device='cuda:0',
       grad_fn=<ClampBackward1>)
tensor([1548, 4810, 4835, 1975])
tensor([2.0377, 0.0000, 0.0000, 6.7428], device='cuda:0',
       grad_fn=<ClampBackward1>)
tensor([6288, 2858, 2064,  749])
tensor([3.0100, 0.7002, 1.1879, 0.0000], device='cuda:0',
       grad_fn=<ClampBackward1>)
ten

In [24]:
print(len(dataset.__getTriplets__()))

1718


In [25]:
torch.save(model.state_dict(), 'model.pth')

In [26]:
# cp "model.pth" "/content/drive/MyDrive/"

In [27]:
# Save The cnn layer of model 
torch.save(model.cnn1.state_dict(), 'cnn1.pth')

# Save The fully connected layer of model
torch.save(model.fc1.state_dict(), 'fc1.pth')


In [28]:
#cp "cnn1.pth"  "/content/drive/MyDrive/"
#cp "fc1.pth"  "/content/drive/MyDrive/"

In [29]:
data_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(256),
    transforms.CenterCrop(224),
   
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [30]:
# define the custom dataset class
class CDATASET(Dataset):
    def __init__(self, dataset_path, transform=None):
        # initialize the list of image paths and labels
        self.image_paths = self.__image_paths__(dataset_path)
        self.labels = self.__getLabels__(dataset_path)
        self.transform = transform



    def __image_paths__(self, dataset_path):
        image_paths = []
        for l in os.listdir(dataset_path):
            path = os.path.sep.join([dataset_path, l])
            image_paths.extend(list(map(lambda x: os.path.sep.join([path, x]), os.listdir(path))))
        return image_paths   

    def __getLabels__(self ,dataset_path):
            labels =[]
            for l in os.listdir(dataset_path):
                path = os.path.sep.join([dataset_path, l])
                for i in range(0 ,len(os.listdir(path))):
                    labels.append(l)
            return labels    

    def __getitem__(self, idx):

        # load the image from disk
        image_path = self.image_paths[idx]
        label = self.labels[idx]
        print(image_path)
        image = io.imread(image_path)
        
        image = torch.from_numpy(np.flip(image,axis=0).copy())
        image = image.numpy()
        # apply the transformations
        if self.transform:
            image = self.transform(image)
        
            

        # return a tuple of the images and the labels
        return image, label

    def __len__(self):
        return len(self.labels)
                


# define the transform(s) to apply to the data
data_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(256),
    transforms.CenterCrop(224),
   
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])







In [31]:
model = SiameseNetwork()
model.cnn1.load_state_dict(torch.load('cnn1.pth'))
model.fc1.load_state_dict(torch.load('fc1.pth'))

cnn = model.cnn1
fc = model.fc1
model = model.to(device)

In [32]:
cnn = model.cnn1
fc = model.fc1

In [33]:
data = CDATASET('data', transform=data_transforms)
dataloader = DataLoader(data, batch_size=4,drop_last=True, shuffle=True, num_workers=0)

In [34]:
import csv
import pandas as pd
ii=0
df=pd.DataFrame(columns=["Name" , "Value"])
dataIter = iter(dataloader)
for image , label in dataIter:
    image = Variable(image)
    image = image.to(device)
    out = cnn(image)
    out=out.view(out.size()[0], -1)
    out = fc(out)
    # Write the output to a csv file with corresponding label
    
    for i in range(0 , len(out)):
       df.loc[ii] = [label[i] , out[i].to('cpu').detach().numpy()]
       ii=ii+1

data/salman/TAM_1172.JPG
data/Usman/WhatsApp Image 2022-06-03 at 10.43.25 AM.jpeg
data/salman/TAM_1171.JPG
data/Usman/WhatsApp Image 2022-06-03 at 10.43.26 AM (1).jpeg
data/Usman/WhatsApp Image 2022-06-03 at 10.43.28 AM.jpeg
data/salman/DSC_0022.JPG
data/salman/TAM_1173.JPG
data/tom cruise/0_Footage-of-terrifying-Tom-Cruise-staring-down-60-Minutes-journalist-Peter-Overton-goes-viral-17-ye.jpg
data/Usman/WhatsApp Image 2022-06-03 at 10.43.26 AM.jpeg
data/salman/DSC_0023.JPG
data/tom cruise/tom-cruise-wants-to-take-after-prince-philip-t.jpg
data/Usman/WhatsApp Image 2022-06-03 at 10.43.27 AM.jpeg


In [35]:
df.to_csv("output.csv")

In [36]:
testData = CDATASET('data', transform=data_transforms)
testData = DataLoader(testData, batch_size=4, shuffle=True, num_workers=0)

In [37]:
d = df.Value.values


In [38]:
ll=[]
lis=[]
orig=[]
pred=[]
correct=0
wrong=0
for image , label in testData:
    image = Variable(image)
    image = image.to(device)
    out = cnn(image)
    out = out.view(out.size(0), -1)
    out = fc(out)
    # Find the top 5 matches using the Euclidean distance from the givens
    for l in range(0 ,len(out)):
        ll=[]
        orig.append(label[l])
        for i in range(0,len(d)):
            dist = PairwiseDistance(2)
            
            dt = torch.Tensor(d[i])
            dt = dt.to(device)
            distt = dist(out[l], dt)
            ll.append(distt)
        lis = torch.Tensor(ll , device='cpu')
        low = 0
        for i in range(len(lis)):
            if lis[i]<lis[0]:
                low=i
        pred.append(df.Name.values[low])

data/salman/DSC_0023.JPG
data/salman/DSC_0022.JPG
data/Usman/WhatsApp Image 2022-06-03 at 10.43.28 AM.jpeg
data/tom cruise/0_Footage-of-terrifying-Tom-Cruise-staring-down-60-Minutes-journalist-Peter-Overton-goes-viral-17-ye.jpg
data/Usman/WhatsApp Image 2022-06-03 at 10.43.26 AM.jpeg
data/salman/TAM_1172.JPG
data/Usman/WhatsApp Image 2022-06-03 at 10.43.28 AM (1).jpeg
data/salman/TAM_1173.JPG
data/tom cruise/Tom_Cruise_by_Gage_Skidmore_2.jpg
data/tom cruise/tom-cruise-wants-to-take-after-prince-philip-t.jpg
data/Usman/WhatsApp Image 2022-06-03 at 10.43.25 AM.jpeg
data/Usman/WhatsApp Image 2022-06-03 at 10.43.26 AM (1).jpeg
data/salman/TAM_1171.JPG
data/Usman/WhatsApp Image 2022-06-03 at 10.43.27 AM.jpeg


In [39]:
print(pred)

['Usman', 'salman', 'tom cruise', 'tom cruise', 'Usman', 'Usman', 'Usman', 'salman', 'tom cruise', 'Usman', 'Usman', 'salman', 'salman', 'Usman']


In [40]:
print(orig)

['salman', 'salman', 'Usman', 'tom cruise', 'Usman', 'salman', 'Usman', 'salman', 'tom cruise', 'tom cruise', 'Usman', 'Usman', 'salman', 'Usman']


In [41]:
for i in range(0,len(orig)):
    if pred[i] == orig[i]:
        correct=correct+1
    else:
        wrong = wrong+1


In [42]:
print(correct)

9


In [43]:
print(wrong)

5


In [44]:
print(lis)

tensor([23.0984, 23.2974, 20.7745, 20.8065, 19.8608, 21.5077, 24.6588, 22.0856,
        26.0341, 21.9722, 21.1380, 20.3738])


In [45]:
# import accuracy score , precision , recall from sklearn
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
print("Accuracy :" , accuracy_score(pred, orig))

Accuracy : 0.6428571428571429
